# Inspecting diaSrc Tables

## Summary
* Loading the data
* Defining quantities such as signal/noise
* Producing the plots for each quantity

In [10]:
import os
import sqlite3

In [11]:
os.environ['SCRATCH']='/global/cscratch1/sd/bos0109'
SCRATCH = %env SCRATCH

In [12]:
from scipy import linalg

In [13]:
import numpy as np

In [14]:
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
%matplotlib inline

In [16]:
from glob import glob

In [17]:
from astropy import time
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import vstack

### Loading the data
 We are loading the data from a set of files already produced and stored.
 The files contain the truth catalog for SN Ia and their parameters, as well
 a HDF5 storage container with the `diaSrc` catalogs stacked as a single table.

In [18]:
truth_lightc = pd.read_csv('../lightcurves/lightcurves_cat_rect_58.0_56.0_-31.0_-32.0.csv')
sntab = pd.read_csv('../catalogs+tables/supernovae_cat_rect_58.0_56.0_-31.0_-32.0.csv')

In [19]:
diaSrc_store = pd.HDFStore(f'{SCRATCH}/results/diaSrc_secrun_fulltables_v5.h5')
diaSrc_store.open()

## diaSrc

In [20]:
dets = diaSrc_store['full_table']

In [ ]:
len(dets)

In [21]:
colnames = dets.columns

In [22]:
fluxes = [acol for acol in colnames if acol.endswith('_instFlux')]

In [23]:
print(fluxes)

['base_SdssShape_instFlux', 'slot_Shape_instFlux', 'base_CircularApertureFlux_3_0_instFlux', 'base_CircularApertureFlux_4_5_instFlux', 'base_CircularApertureFlux_6_0_instFlux', 'base_CircularApertureFlux_9_0_instFlux', 'base_CircularApertureFlux_12_0_instFlux', 'slot_ApFlux_instFlux', 'base_CircularApertureFlux_17_0_instFlux', 'base_CircularApertureFlux_25_0_instFlux', 'base_CircularApertureFlux_35_0_instFlux', 'base_CircularApertureFlux_50_0_instFlux', 'base_CircularApertureFlux_70_0_instFlux', 'base_GaussianFlux_instFlux', 'base_PeakLikelihoodFlux_instFlux', 'base_PsfFlux_instFlux', 'slot_PsfFlux_instFlux', 'ip_diffim_NaiveDipoleFlux_pos_instFlux', 'ip_diffim_NaiveDipoleFlux_neg_instFlux', 'ip_diffim_PsfDipoleFlux_pos_instFlux', 'ip_diffim_PsfDipoleFlux_neg_instFlux', 'ip_diffim_DipoleFit_pos_instFlux', 'ip_diffim_DipoleFit_neg_instFlux', 'ip_diffim_DipoleFit_instFlux', 'ip_diffim_forced_PsfFlux_instFlux']


In [24]:
for aflux in fluxes:
    try:
        dets[aflux+'_SN'] = dets[aflux]/dets[aflux+'Err']
    except:
        print(aflux)

ip_diffim_DipoleFit_instFlux


In [25]:
def eigenvals_inertia(Ixx, Iyy, Ixy):
    delta = (Iyy - Ixx)**2 + 4*Ixy*Ixy
    
    Ixx_P = Ixx + Iyy + np.sqrt(delta)
    Iyy_P = Ixx + Iyy - np.sqrt(delta)
    return(Ixx_P, Iyy_P)

In [26]:
secmoments = [acol for acol in colnames if acol.endswith('_xx')]

In [27]:
for mom in secmoments:
    generic_name = mom.strip('_xx')
    print('trying:', generic_name)
    Ixx = dets[mom].values
    Iyy = dets[generic_name+'_yy'].values
    try:
        Ixy = dets[generic_name+'_xy'].values
        dets[generic_name+'_moments_ratio_qs'] = Ixx/Iyy
        dets[generic_name+'_moments_quad'] = Ixx**2 + Iyy**2
        try:
            Ixxp, Iyyp = eigenvals_inertia(Ixx, Iyy, Ixy)
        
            dets[generic_name+'_moments_diag_xx'] = Ixxp
            dets[generic_name+'_moments_diag_yy'] = Iyyp
            dets[generic_name+'_moments_diag_ratio'] = Ixxp/Iyyp
            dets[generic_name+'_moments_diag_quad'] = Ixxp**2 + Iyyp**2
        except:
            print('failed diag:', generic_name)
    except:
        print('failed complete:', generic_name)
        


trying: base_SdssShape
trying: slot_Shape
trying: base_SdssShape_psf
trying: slot_PsfShape
trying: slot_Shape_psf


In [28]:
diaSrc_store['full_table'] = dets
diaSrc_store.flush()

## Forced

In [29]:
dets = diaSrc_store['full_table_forced']

In [ ]:
len(dets)

In [30]:
colnames = dets.columns

In [31]:
fluxes = [acol for acol in colnames if acol.endswith('_instFlux')]

In [32]:
print(fluxes)

['base_SdssShape_instFlux', 'slot_Shape_instFlux', 'base_LocalBackground_instFlux', 'base_PsfFlux_instFlux']


In [33]:
for aflux in fluxes:
    try:
        dets[aflux+'_SN'] = dets[aflux]/dets[aflux+'Err']
    except:
        print(aflux)

In [34]:
secmoments = [acol for acol in colnames if acol.endswith('_xx')]

In [35]:
for mom in secmoments:
    generic_name = mom.strip('_xx')
    print('trying:', generic_name)
    Ixx = dets[mom].values
    Iyy = dets[generic_name+'_yy'].values
    Ixy = dets[generic_name+'_xy'].values
    try:
        dets[generic_name+'_moments_ratio_qs'] = Ixx/Iyy
        try:
            Ixxp, Iyyp = eigenvals_inertia(Ixx, Iyy, Ixy)
        
            dets[generic_name+'_moments_xx_diag'] = Ixxp
            dets[generic_name+'_moments_yy_diag'] = Iyyp
            dets[generic_name+'_moments_ratio_diag'] = Ixxp/Iyyp
        except:
            print('failed diag:', generic_name)
    except:
        print('failed complete:', generic_name)
        


trying: base_SdssShape
trying: slot_Shape
trying: base_SdssShape_psf
trying: slot_PsfShape
trying: slot_Shape_psf


In [37]:
len(dets)

41757617

In [39]:
import gc
gc.collect()

72

In [38]:
diaSrc_store['full_table_forced'] = dets
diaSrc_store.flush()

MemoryError: 

In [40]:
diaSrc_store.close()